Approach:
1. Business model is to make one time purchase (buy real estate) and gradually recover costs through rental income
2. Zip codes with the least Break even time (our profit metric) will be the most lucrative investments
3. To compute break even time we require 2 facts per zip code i.e. current estate price & current rent (daily/weekly/monthly)
4. Current estate price can be sourced from respective 'month-of-year' column in Zillow data, while 'price' column in AirBnb data indicates current rental prices
5. Note that AirBnb data is at property level while Zillow data is at zip code level, so we need to aggregate AirBnb data by zip code before joining/mapping real estate values to respective rental prices
6. To minimize rental data loss while merging datasets, 2 bed rent may be estimated from 1 bed or studio rent by considering average sq. ft. ratio of 2 bed to 1 bed or studio respectively (assumption 4); average sq. ft. for 2,1,0 beds in US can be obtained from 3rd party website like <a href='https://www.rentcafe.com/'>rentcafe.com</a>


Assumptions:
1. Rental property occupancy rate is 75%
2. The time value of money discount rate is 0%
3. No mortgage/interest rate is charged when buying real estate properties
4. A 1000 sq. ft. property in Bronx or Manhattan generates 2x the revenue and costs 2x as any other 500 sq. ft. property within that same locale
5. All 2 bedroom property types (Apartment, House, Loft, Condominium, etc.) have equal rent
6. All 2 bedroom properties have equal sq. feet area

In [679]:
#import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime

In [680]:
#input AirBnb data
rentals=pd.read_csv('listings.csv')
#input Zillow data
estate=pd.read_csv('Zip_Zhvi_2bedroom.csv')

c:\program files (x86)\python\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [681]:
#assume current date as May 1, 2017; can be changed to use the actual date when working with fresh data
curr_date='2017-05-01'
curr_date=datetime.strptime(curr_date, '%Y-%m-%d')
#extract current month-of-year from any given date
curr_month=curr_date.month
print(curr_month)
curr_year=curr_date.year
print(curr_year)

5
2017


In [682]:
rentals.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [683]:
#we consider only current month's rentals, i.e. May 2017 listings
rentals['last_scraped_month']=pd.to_datetime(rentals['last_scraped']).dt.month
curr_rentals=rentals[rentals['last_scraped_month']==curr_month]

In [684]:
print(rentals.shape[0])
#~40k total listings
print((curr_rentals['host_location'].value_counts(normalize=True).head(10)*100).astype(int))

40753
New York, New York, United States           80
US                                          10
Los Angeles, California, United States       0
San Francisco, California, United States     0
Paris, Île-de-France, France                 0
United States                                0
Brooklyn, NY                                 0
Brooklyn, New York, United States            0
Boston, Massachusetts, United States         0
London, England, United Kingdom              0
Name: host_location, dtype: int32


80% of rental host locations contain the string 'New York, New York, United States'

In [685]:
#filter on locations within NYC
flag=curr_rentals['host_location'].str.contains('New York, New York|Brooklyn|Bronx|Queens|Manhattan|Staten')
curr_nyc_rentals=curr_rentals[flag==True]
print(curr_nyc_rentals.shape[0])
#~32k NYC listings

32706


In [686]:
(curr_nyc_rentals['bedrooms'].value_counts(normalize=True)*100).round()

1.0     75.0
2.0     12.0
0.0      8.0
3.0      4.0
4.0      1.0
5.0      0.0
6.0      0.0
7.0      0.0
8.0      0.0
10.0     0.0
Name: bedrooms, dtype: float64

75% rental listings are 1 beds, 12% are 2 beds, 8% are studios (0 beds) while remaining variations comprise 5%.

We will filter out remaining variations for now.

However instead of focusing only on 2 beds and losing 75+8=83% data, we estimate 2 bed rent from 1 and 0 bed listings.

This will provide better support/validity to our end results.

Since sq. ft. is missing from most rental listings, we use mean sq. ft. estimates for US homes 

(courtesy: https://www.rentcafe.com/blog/rental-market/us-average-apartment-size-trends-downward/).

Thus we have on average across US (approximate values),  

2 bed=1100 sq. ft.

1 bed=750  sq. ft.

0 bed=500  sq. ft.

Consequently, 

2 bed rent = (1100/750) * 1 bed rent,

2 bed rent = (1100/500) * 0 bed rent

Simplifying, 

2 bed rent = 1.47 * 1 bed rent,

2 bed rent = 2.20 * 0 bed rent

Thus using above formula we compute a common derived column for 2,1 & 0 bed variations.


In [687]:
curr_nyc_rentals=curr_nyc_rentals[curr_nyc_rentals['bedrooms'].isin(['2', '1', '0'])]
print(curr_nyc_rentals.shape[0])
#~31k NYC 2,1,0 bedroom listings

31101


In [688]:
curr_nyc_rentals[['id', 'zipcode', 'price', 'weekly_price', 'monthly_price']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31101 entries, 0 to 40743
Data columns (total 5 columns):
id               31101 non-null int64
zipcode          30685 non-null object
price            31101 non-null object
weekly_price     7602 non-null object
monthly_price    6259 non-null object
dtypes: int64(1), object(4)
memory usage: 971.9+ KB


Zip code has 416/31101 = 1.3% missing values.

Price (nightly_price) has no missing values.

Weekly_price has 23499/31101 = 75.6% missing values.

Monthly_price has 24842/31101 = 79.9% missing values.

Maybe nightly_price is mandatory field while weekly and monthly pricing are optional.

Thus we ignore weekly and monthly prices in further analysis.


In [689]:
#list top 10 NYC zip code count
print(curr_nyc_rentals.zipcode.value_counts().head(10))
print(curr_nyc_rentals.zipcode.value_counts().tail(10))

11211      1674
11206      1152
11221      1054
10002       953
11216       912
10009       869
11238       841
11222.0     739
11237       690
10025       652
Name: zipcode, dtype: int64
11433         1
11426-1175    1
10281         1
11103-3233    1
10080         1
11427.0       1
10078         1
11385-2308    1
10119         1
10279         1
Name: zipcode, dtype: int64


Zip codes have floating values, some values in extended format and some garbage values.

In [690]:
#verify uniqueness of AirBnb listing id column
curr_nyc_rentals.duplicated('id').sum()

0

In [691]:
#restrict zip code length to max. 5 digits and truncate trailing characters 
curr_nyc_rentals['zipcode2']=curr_nyc_rentals.zipcode.str[0:5]
#restrict zip code length to just 5 digits and filter garbage strings 
curr_nyc_rentals=curr_nyc_rentals[curr_nyc_rentals['zipcode2'].str.len()==5]
print(curr_nyc_rentals.shape[0])
#~24.5k listings with valid NYC zip code

24553


In [692]:
#check missing values in zipcode2 column
curr_nyc_rentals['zipcode2'].isnull().sum()

0

In [693]:
#convert zipcode2 column from string to numeric type
curr_nyc_rentals['zipcode2']=pd.to_numeric(curr_nyc_rentals['zipcode2'])
#list top 10 NYC zip code count
print(curr_nyc_rentals.zipcode2.value_counts().head(10))

11211    1674
11206    1152
11221    1054
10002     953
11216     912
10009     869
11238     841
11237     690
10025     652
10011     619
Name: zipcode2, dtype: int64


In [694]:
#convert currency to plain numeric type by removing $ symbol and comma separator
curr_nyc_rentals['price']=pd.to_numeric(curr_nyc_rentals['price'].replace('[\$,]', '', regex=True))
#we now normalize all listing prices to 2 bedroom apartment prices
#create derived column 'effective price' such that 
#  2 bed rent = 1.47 * 1 bed rent, 
#  2 bed rent = 2.20 * 0 bed rent
curr_nyc_rentals['effective_price']=curr_nyc_rentals['price']
curr_nyc_rentals.loc[curr_nyc_rentals['bedrooms'] == 1, 'effective_price']=1.47*curr_nyc_rentals['price']
curr_nyc_rentals.loc[curr_nyc_rentals['bedrooms'] == 0, 'effective_price']=2.2*curr_nyc_rentals['price']

In [695]:
#check count of unique zip codes in NYC
print(curr_nyc_rentals.zipcode.value_counts().shape[0])

181


We have rental listings from 181 unique NYC zip codes

In [696]:
#verify uniqueness of Zillow zip code column
estate.duplicated('RegionName').sum()

0

In [697]:
estate.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02',
       '2017-03', '2017-04', '2017-05', '2017-06'],
      dtype='object', length=262)

'Month-of-year' series begins after first 7 columns.

'Month-of-year' columns are arranged in ascending order of time from left to right.

In [698]:
#function to input Zillow estate dataframe, current date string and output respective column index

def find_estate_month_col_index(df, curr_date_str):
        
    #initialize necessary variables
    curr_estate_index=-1
    #extract month, year from a date string
    curr_date=datetime.strptime(curr_date_str, '%Y-%m-%d')
    curr_year=curr_date.year
    curr_month=curr_date.month
    
    #start matching month & year from the rightmost column till the 8th column from left
    for curr_estate_index in range(df.shape[1]-1,6,-1):
        if(datetime.strptime(df.columns[curr_estate_index], '%Y-%m').year == curr_year and
           datetime.strptime(df.columns[curr_estate_index], '%Y-%m').month == curr_month):
            #found required column index
            return curr_estate_index
        
    #column index not found
    return curr_estate_index

#assume current date as May 1, 2017; can be changed to use the actual date when working with fresh data
curr_estate_index=find_estate_month_col_index(estate, '2017-05-01')
curr_estate_index
#we select the 261th or second last column!

260

In [699]:
#retain only required columns (region name, city & estate price) from Zillow data
curr_estate=estate.iloc[:,[1,2,curr_estate_index]]
curr_estate.columns

Index(['RegionName', 'City', '2017-05'], dtype='object')

In [700]:
curr_estate.info()
#no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8946 entries, 0 to 8945
Data columns (total 3 columns):
RegionName    8946 non-null int64
City          8946 non-null object
2017-05       8946 non-null int64
dtypes: int64(2), object(1)
memory usage: 174.8+ KB


In [701]:
print(curr_estate.shape[0])
#~9k total zip codes
#filter on NYC zip codes
curr_nyc_estate=curr_estate[curr_estate['City']=='New York']
print(curr_nyc_estate.shape[0])

8946
25


Real estate data contains just 25 NYC zip codes, other prominent NYC regions are absent!

Although we have rental data for 181 zip codes, we can retain only 25 of them as estate prices are absent for remaining zip codes.

However we cannot avoid this data loss, so just gotta live with it.

In [702]:
#rename rightmost column
curr_nyc_estate=curr_nyc_estate.rename(columns={curr_nyc_estate.columns[curr_nyc_estate.shape[1]-1]:'curr_estate_price'})
curr_nyc_estate.columns

Index(['RegionName', 'City', 'curr_estate_price'], dtype='object')

In [703]:
#join estate price data with rental listing data based on zip code
combined=pd.merge(curr_nyc_estate, curr_nyc_rentals[['zipcode2', 'effective_price']], left_on='RegionName', right_on='zipcode2')
combined=combined.rename(columns={'curr_estate_price':'estate_price', 'effective_price':'nightly_rent'})
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5626 entries, 0 to 5625
Data columns (total 5 columns):
RegionName      5626 non-null int64
City            5626 non-null object
estate_price    5626 non-null int64
zipcode2        5626 non-null int64
nightly_rent    5626 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 241.7+ KB


None of the columns have missing values.

In [704]:
#group by zipcode2, compute row count & median prices, sort in descending order of row count 
#median is preferred to mean in order to limit impact of outliers
zip_summary=combined[['estate_price', 'zipcode2', 'nightly_rent']].groupby('zipcode2').agg(['count','median']).sort_values(by=[('estate_price', 'count')], ascending=False)
zip_summary

estate_price          nightly_rent         
                count   median        count   median
zipcode2                                            
10025             652  1390000          652  161.700
10011             619  2419700          619  286.650
10003             596  2109100          596  220.500
10014             554  2498400          554  294.000
11215             530  1066200          530  161.700
10023             374  2095000          374  275.000
11201             342  1407300          342  199.500
10128             331  1720500          331  199.000
10028             286  2015700          286  200.000
10013             273  3262200          273  257.250
11217             251  1276400          251  176.400
11231             230  1211700          230  175.000
10021             215  1774100          215  220.500
10022             204  1997800          204  237.500
10036              49  1717100           49  257.250
11234              27   477100           27  132.300
10304              26   321300           26   86.730
10305              20   418900           20  107.920
11434              16   380200           16   86.730
10306              11   347800           11  116.000
10314               8   352300            8  109.515
10312               7   351700            7   70.560
10303               4   322100            4   80.115
10308               1   402900            1  139.650

In [705]:
#to eliminate weak groups, filter out zip codes having less than 50 data points for nightly rent
zip_summary2=zip_summary[zip_summary[('nightly_rent', 'count')] >= 50][[('estate_price', 'median'), ('nightly_rent', 'median')]].reset_index()
#flatten summary table, drop intermediate column level
zip_summary2.columns=zip_summary2.columns.droplevel(1)
#rename columns
zip_summary2=zip_summary2.rename(columns={'zipcode2':'Zip code', 'estate_price':'Estate price ($)', 'nightly_rent':'Median nightly rent ($)'})
zip_summary2

,Zip code,Estate price ($),Median nightly rent ($)
0,10025,1390000,161.70
1,10011,2419700,286.65
2,10003,2109100,220.50
3,10014,2498400,294.00
4,11215,1066200,161.70
5,10023,2095000,275.00
6,11201,1407300,199.50
7,10128,1720500,199.00
8,10028,2015700,200.00
9,10013,3262200,257.25


Break even calculation:

Assume expected occupancy factor of 0.75.

Assume discount rate for rent as 0%, thus rent amounts received in future are as valuable as the present amount.

Thus, 

Break even time = (Estate price) / (Total estimated rent * Expected occupancy)

In [706]:
#round up break even time to get complete time periods
#convert all columns to integer format and get rid of decimal places
zip_summary2['Max. break even time (months)']=np.ceil(zip_summary2['Estate price ($)']/(zip_summary2['Median nightly rent ($)']*0.75*30)).astype(int)
zip_summary2['Max. break even time (years)']=np.ceil(zip_summary2['Estate price ($)']/(zip_summary2['Median nightly rent ($)']*0.75*30*12)).astype(int)
zip_summary2=zip_summary2.round({'Median nightly rent ($)':0, 'Estate price ($)':0}).astype(int)
#format estate prices with thousands separator
zip_summary2['Estate price ($)']=zip_summary2.apply(lambda x: "{:,}".format(x['Estate price ($)']), axis=1)
#sort rows in ascending order of break even time
zip_summary2=zip_summary2.sort_values(['Max. break even time (months)']).reset_index(drop=True)
zip_summary2

,Zip code,Estate price ($),Median nightly rent ($),Max. break even time (months),Max. break even time (years)
0,11215,"1,066,200",162,294,25
1,11231,"1,211,700",175,308,26
2,11201,"1,407,300",200,314,27
3,11217,"1,276,400",176,322,27
4,10023,"2,095,000",275,339,29
5,10021,"1,774,100",220,358,30
6,10022,"1,997,800",238,374,32
7,10011,"2,419,700",287,376,32
8,10014,"2,498,400",294,378,32
9,10025,"1,390,000",162,383,32


Above table displays all zip codes in descending order of Profit & ascending order of Break even time.

Zip codes 11215 and 11231 have lower break-even times, and hence are more profitable.

If selecting between zip codes with similar break even times (for ex. 11201 & 11217), we choose the smaller initial investment (in this case, 11217).

Next steps:

1. Consider historical trend in estate time series and accordingly vary future median rental income per zip code. This would basically increase/decrease rental income over months in accordance with changes in mean real estate price. Here we are assuming that rent is correlated to real estate price.

2. Consider weekly/monthly rental prices whenever available and obtain more robust break even estimates. Weekly and monthly rent is cheaper than nightly rent, so considering longer terms would give us the worst-case scenario.

3. Vary occupancy rate for each listing using availability data for next 30, 60, 90 and 365 days. Here we are additionally considering that host may be unable to rent, apart from whether he/she gets a prospect or not. Currently AirBnb disables host calendar for 2 reasons - host does not wish to rent, or someone already booked. But in this case, we are ignoring the 2nd reason as we don't have data to distinguish between the two. 